In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import time
import collections
import random
import pandas as pd
import datetime
import scipy as sci
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
import torch



#**General form**

In [ ]:

r0=2
v0=2

lambda1 =  2.689E-01
lambda2 = 1.450E-02
lambda3 = -3.991E-01
lambda4 = 1.939E-01
power1 = 1.552E-01
power2 = 7.777E-02
power3 = 2.298E-01

lambda1_v = 0
lambda2_v = 0
lambda3_v = 0
lambda4_v = 0
power1_v = 0
power2_v = 0
power3_v = 0

lambda1_2 =  -1.294E-01
lambda2_2 = 5.804E-01
lambda3_2 = -7.528E-01
lambda4_2 = 4.247E-01
power1_2 = -7.914E-02
power2_2 = 5.482E-02
power3_2 = 1.512E-01

lambda1_v_2 = 0
lambda2_v_2 = 0
lambda3_v_2 = 0
lambda4_v_2 = 0
power1_v_2 = 0
power2_v_2 = 0
power3_v_2 = 0
int_range = 0



def total_infer_x(s,i):
    n_array = np.array(n)
    self_index=np.repeat(i,n-1)
    
    others_index=list(range(n))
    others_index.remove(i)
    
    
    x1=np.take(s, self_index)
    y1=np.take(s, self_index + n_array)
    x2=np.take(s, others_index)
    y2=np.take(s, others_index + n_array)
    dist=((x1-x2)**2 + (y1-y2)**2)**(1/2)
    

    vi_x=np.take(s, 2*n_array + self_index) #self
    vi_y=np.take(s, 3*n_array + self_index) #self
    vj_x=np.take(s, 2*n_array + others_index)
    vj_y=np.take(s, 3*n_array + others_index)
    vel_rel=((vi_x-vj_x)**2 + (vi_y-vj_y)**2)**(1/2)
    
    local_dist_func = np.heaviside(int_range - dist,1)
    
    if i<20:
        F_int_x=(x1-x2)/dist*(lambda1*(dist**power1)+lambda2*(dist**power2)+lambda3*(dist**power3)+lambda4) + \
        (vi_x-vj_x)/vel_rel*(lambda1_v*(dist**power1_v)+lambda2_v*(dist**power2_v)+lambda3_v*(dist**power3_v)+lambda4_v)*local_dist_func
        total_interaction_x=sum(F_int_x)
        
    else:
        F_int_x = (x1-x2)/dist*(lambda1_2*(dist**power1_2)+lambda2_2*(dist**power2_2)+lambda3_2*(dist**power3_2)+lambda4_2) + \
        (vi_x-vj_x)/vel_rel*(lambda1_v_2*(dist**power1_v_2)+lambda2_v_2*(dist**power2_v_2)+lambda3_v_2*(dist**power3_v_2)+lambda4_v_2)*local_dist_func
        
        if int_range == 0:
            total_interaction_x=sum(F_int_x)
        else:
            total_interaction_x=sum(F_int_x) + np.random.normal(0,10)


    v_i_x=s[2*n+i]
    v_i_y=s[3*n+i]
    v_i_mag = (v_i_x**2+v_i_y**2)**(1/2)
    self_propel_x = 1*(v_i_x/v_i_mag)*(v0-v_i_mag)
    
    return self_propel_x+total_interaction_x

def total_infer_y(s,i):
    n_array = np.array(n)
    self_index=np.repeat(i,n-1)
    
    others_index=list(range(n))
    others_index.remove(i)
    
    x1=np.take(s, self_index)
    y1=np.take(s, self_index + n_array)
    x2=np.take(s, others_index)
    y2=np.take(s, others_index + n_array)
    dist=((x1-x2)**2 + (y1-y2)**2)**(1/2)
    

    vi_x=np.take(s, 2*n_array + self_index) #self
    vi_y=np.take(s, 3*n_array + self_index) #self
    vj_x=np.take(s, 2*n_array + others_index)
    vj_y=np.take(s, 3*n_array + others_index)
    vel_rel=((vi_x-vj_x)**2 + (vi_y-vj_y)**2)**(1/2)
    
    local_dist_func = np.heaviside(int_range - dist,1)
    
    if i<20:
        F_int_y = (y1-y2)/dist*(lambda1*(dist**power1)+lambda2*(dist**power2)+lambda3*(dist**power3)+lambda4) + \
        (vi_y-vj_y)/vel_rel*(lambda1_v*(dist**power1_v)+lambda2_v*(dist**power2_v)+lambda3_v*(dist**power3_v)+lambda4_v)*local_dist_func
        total_interaction_y=sum(F_int_y)
        
    else:
        F_int_y = (y1-y2)/dist*(lambda1_2*(dist**power1_2)+lambda2_2*(dist**power2_2)+lambda3_2*(dist**power3_2)+lambda4_2) + \
        (vi_y-vj_y)/vel_rel*(lambda1_v_2*(dist**power1_v_2)+lambda2_v_2*(dist**power2_v_2)+lambda3_v_2*(dist**power3_v_2)+lambda4_v_2)*local_dist_func
        
        if int_range == 0:
            total_interaction_y=sum(F_int_y)
        else:
            total_interaction_y=sum(F_int_y) + np.random.normal(0,10)
    
    v_i_x=s[2*n+i]
    v_i_y=s[3*n+i]
    v_i_mag = (v_i_x**2+v_i_y**2)**(1/2)
    self_propel_y = 1*(v_i_y/v_i_mag)*(v0-v_i_mag)

    return self_propel_y+total_interaction_y


def F_infer_ivp(t,s):
    return np.concatenate((s[2*n:3*n],s[3*n:4*n],[total_infer_x(s,i) for i in range(n)], [total_infer_y(s,i) for i in range(n)]))


In [ ]:
##########Inital condition ##############

position_list_x=[]
position_list_y=[]
velocity_list_x=[]
velocity_list_y=[]
random.seed(131)
field = 1
# random.seed()
for i in range(40):
    position_list_x.append(random.uniform(-field, field))
    position_list_y.append(random.uniform(-field, field))
    velocity_list_x.append(random.uniform(-1,1))
    velocity_list_y.append(random.uniform(-1,1))

eps = torch.finfo(torch.float32).eps

nodes_G = torch.tensor([position_list_x, position_list_y, velocity_list_x, velocity_list_y], dtype=torch.float32)
nodes_np=nodes_G.to('cpu').detach().numpy()

s0=nodes_np.reshape(-1)
n=nodes_np.shape[1]

In [ ]:
dt=0.1
T=200
t=np.arange(0,T+dt,dt)

solution=solve_ivp(F_infer_ivp,[0,T],s0, max_step=0.01, method='RK45', t_eval=t)

print(solution)


In [ ]:
import matplotlib.animation as animation

solution_plot=np.transpose(solution.y)

speed = 2 ####play spped####
t_plot=np.arange(0,T+dt*speed ,dt*speed)
h = solution_plot[:,0:2*n]
velocity=solution_plot[:,2*n:4*n]

fig = plt.figure(figsize=(8, 8))
ax = plt.axes()

tail_length = 10

############################### order parameter #####################
velocity_x = velocity[:,0:n]
velocity_y = velocity[:,n:]
center_x = np.mean(h[:,0:n],axis=1,keepdims=True)
center_y = np.mean(h[:,n:],axis=1,keepdims=True)
r_x, r_y = h[:,0:n]-center_x, h[:,n:]-center_y

r_mag = np.sqrt(np.square(r_x)+np.square(r_y))
vel_mag=np.sqrt(np.square(velocity_x)+np.square(velocity_y)) #N X V
vel_mag_sum=np.sum(vel_mag, axis=1)

vel_x_sum = np.sum(velocity_x, axis=1)
vel_y_sum = np.sum(velocity_y, axis=1)

vel_vec_mag = np.sqrt(np.square(vel_x_sum)+np.square(vel_y_sum)) # abs(sum of velocity vector)

order_para = vel_vec_mag/vel_mag_sum

ang_mom = r_x*velocity_y - r_y * velocity_x

denom = np.sum(r_mag*vel_mag, axis=1)
ang_mom_sum = np.abs(np.sum(r_x*velocity_y - r_y * velocity_x, axis=1))
ang_mom_abs = np.sum(np.abs(r_x*velocity_y - r_y * velocity_x), axis=1)

order_para_rot = ang_mom_sum/denom
order_para_rot_abs = ang_mom_abs/denom

##########################################################################
init_color = np.array([0.,0.,0.])
cw_color =  np.array([0.,100.,250.])
ccw_color =  np.array([240.,84.,0.])
flock_color = np.array([145.,84.,192.])
agent_color = np.array([105./255,105./255,105./255,0.8])
agent_color_2 = np.array([181./255,181./255,182./255,0.8])


def animate(i):
    window = 7.5
    spl=20
    ax.clear()
    
    ax.scatter(h[i*speed,0:spl],h[i*speed,n:n+spl],color=agent_color,s=200, zorder=0)
    for j in range(spl):
        ax.plot(h[i*speed - tail_length:i*speed+1,j],h[i*speed- tail_length:i*speed+1,n+j], '--', color=agent_color, lw=1, ms=1)
    
    ax.scatter(h[i*speed,spl:n],h[i*speed,n+spl:],marker = 'D',color=agent_color_2,s=200, zorder=0)
    for j in range(spl,n):
        ax.plot(h[i*speed - tail_length:i*speed+1,j],h[i*speed- tail_length:i*speed+1,n+j], '--', color=agent_color_2, lw=1, ms=1)
    
    ################ coloring ######################
    rot_color = []
    if int_range == 0:
        for ii in range(len(ang_mom[i*speed,:])):
            if ang_mom[i*speed,ii]>0 and order_para_rot_abs[i*speed]>0.7:
                rot_color.append(np.append(ccw_color*(order_para_rot_abs[i*speed]-0.7)/0.3/255, [0.8]))
            elif ang_mom[i*speed,ii]<0 and order_para_rot_abs[i*speed]>0.7:
                rot_color.append(np.append(cw_color*(order_para_rot_abs[i*speed]-0.7)/0.3/255, [0.8]))
            else:
                rot_color.append(np.append(init_color,[0.8]))
    else:
        for ii in range(len(ang_mom[i*speed,:])):
            if order_para[i*speed]>0.7:
                rot_color.append(np.append(flock_color*(order_para[i*speed]-0.7)/0.3/255, [0.8]))
            else:
                rot_color.append(np.append(init_color,[0.8]))
    ##################################################

    origin=np.array([h[i*speed,0:n], h[i*speed,n:]])
    plt.quiver(*origin, velocity[i*speed,0:n], velocity[i*speed,n:], scale_units='xy', angles='xy', color=rot_color, \
               scale=2, width=0.006, headwidth=5,zorder=2)

    center_x_i = np.mean(h[i*speed,0:n])
    center_y_i = np.mean(h[i*speed,n:])
    
    ax.set_title('Time: {:.2f}'.format(t[i*speed]))
    ax.set_xticks(np.arange(-100, 100, 5))
    ax.set_yticks(np.arange(-100, 100, 5))
    ax.set_xlim(center_x_i - window, center_x_i +window)
    ax.set_ylim(center_y_i - window, center_y_i +window)

anim = animation.FuncAnimation(fig, animate, frames=len(t_plot))


from IPython.display import HTML
HTML(anim.to_html5_video())